<a href="https://colab.research.google.com/github/Gogulaanand/Fault-Diagnosis/blob/master/Fault%20Diagnosis_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import io
import tensorflow as tf


In [2]:
from google.colab import files
uploaded = files.upload()

Saving projdata.csv to projdata.csv


In [0]:
df=pd.read_csv(io.StringIO(uploaded['projdata.csv'].decode("utf-8")),engine='python',sep=",")
ds=df.sample(frac=1)


In [0]:
y=np.array(df[['Fault']])
X=ds.as_matrix(columns=df.columns[1:])
X=X.T
y=y.T
X=tf.constant(X,name="X",dtype=tf.float64)
y=tf.constant(y,name="y",dtype=tf.float64)


In [5]:
nx=np.min(np.shape(X))
ny=np.min(np.shape(y))
nh=3
nl=2
print("Size of input layer: "+str(nx))
print("Size of output layer: "+str(ny))
print("Size of hidden units in each layer: "+str(nh))
print("No of hidden layers: "+str(nl))


Size of input layer: 3
Size of output layer: 1
Size of hidden units in each layer: 3
No of hidden layers: 2


In [0]:
def sigmoid(z):
  
  x=tf.placeholder(tf.float64,name="x")
  
  sigmoid=tf.sigmoid(x)
  
  with tf.Session() as sess:
    result=sess.run(sigmoid,feed_dict={x:z})
  sess.close()
  
  return result

In [0]:
def initialize_parameters():
  
  with tf.variable_scope("initialize_parameters",reuse=tf.AUTO_REUSE):
    
  
    W1 = tf.get_variable("W1",[3,3],dtype=tf.float64,initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.get_variable("b1",[3,1],dtype=tf.float64,initializer=tf.zeros_initializer())
    W2 = tf.get_variable("W2",[3,3],dtype=tf.float64,initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.get_variable("b2",[3,1],dtype=tf.float64,initializer=tf.zeros_initializer())
    W3 = tf.get_variable("W3",[1,3],dtype=tf.float64,initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.get_variable("b3",[1,1],dtype=tf.float64,initializer=tf.zeros_initializer())
    

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
    return parameters

In [0]:
def forward_propagation(X, parameters):
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3']
    
    
    Z1 = tf.add(tf.matmul(W1,X),b1)                                              # Z1 = np.dot(W1, X) + b1
    A1 = tf.nn.relu(Z1)                                              # A1 = relu(Z1)
    Z2 = tf.add(tf.matmul(W2,A1),b2)                                              # Z2 = np.dot(W2, a1) + b2
    A2 = tf.nn.relu(Z2)                                              # A2 = relu(Z2)
    Z3 = tf.add(tf.matmul(W3,A2),b3)                                              # Z3 = np.dot(W3,Z2) + b3
    
    return Z3
    

In [0]:
def compute_cost(Z3, Y):
    
    
    
    logits = tf.transpose(Z3)
    labels = tf.transpose(Y)
    
    
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,labels=labels))
    
    
    return cost

In [0]:
def random_mini_batches(X,y,mini_batch_size=128):
  
  m=y.shape[1]
  
  
  
  #permutation=list(np.random.permutation(int(m)))      #to shuffle the x and y matrices
  #print(permutation)
  #shuffled_X=X[:,permutation]
  #shuffled_Y=y[:,permutation].reshape((1,m))
  shuffled_X=X
  shuffled_Y=y

  mini_batches=[]
  
  total_possible_mini_batches=int(math.floor(int(m)/int(mini_batch_size)))
  
  for k in range(total_possible_mini_batches):
    
    mini_batch_X=shuffled_X[:,(mini_batch_size*k):(mini_batch_size*(k+1))]
    mini_batch_Y=shuffled_Y[:,(mini_batch_size*k):(mini_batch_size*(k+1))]
    
    mini_batch=(mini_batch_X,mini_batch_Y)
   
    mini_batches.append(mini_batch)
    
  
  if m%mini_batch_size!=0:
    
    mini_batch_X=shuffled_X[:,total_possible_mini_batches*mini_batch_size:m]
    mini_batch_Y=shuffled_Y[:,total_possible_mini_batches*mini_batch_size:m]
   
    mini_batch=(mini_batch_X,mini_batch_Y)
    mini_batches.append(mini_batch)
  
    
  return mini_batches

In [0]:
def model(X,y, learning_rate = 0.0005,num_epochs = 50, minibatch_size = 64, print_cost = True):
  
    
    
    with tf.variable_scope("model",reuse=tf.AUTO_REUSE):              
      costs = []                                       

      m=y.shape[1]
      parameters = initialize_parameters()

      Z3 = forward_propagation(X,parameters)

      cost = compute_cost(Z3,y)

      optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

      init = tf.global_variables_initializer()

      with tf.Session() as sess:

          sess.run(init)

          for epoch in range(num_epochs):

              epoch_cost = 0                       
              num_minibatches = m // minibatch_size 
              minibatches = random_mini_batches(X,y,minibatch_size)

              for minibatch in minibatches:




                  (minibatch_X, minibatch_Y) = minibatch
                  
                  x=tf.placeholder(tf.float64,shape=minibatch_X.shape,name="x")
                  y1=tf.placeholder(tf.float64,shape=minibatch_Y.shape,name="y")


                  _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={x: minibatch_X.eval(), y1: minibatch_Y.eval()})


                  epoch_cost += int(minibatch_cost) / int(num_minibatches)


              if print_cost == True:
                  print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
                  costs.append(epoch_cost)


          plt.plot(np.squeeze(costs))
          plt.ylabel('cost')
          plt.xlabel('iterations (per tens)')
          plt.title("Learning rate =" + str(learning_rate))
          plt.show()


          parameters = sess.run(parameters)
   
          correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(y))
          accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
          print ("Train Accuracy:", accuracy.eval({X: X, y: y}))
          #print ("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
    return parameters

In [0]:

parameters = model(X,y)